# Contexto
- Spotify ya no provee de acceso a ciertos api Points desde Noviembre del 2024
- Encontré en https://gist.github.com/andytlr/4104c667a62d8145aa3a el siguiente comentario:
    - Oh, I found a better list really, I found this chosic site that splits it into genres and subgenres and I extracted it into a json file that I'll attach here (https://drive.google.com/file/d/1w7H_myVKMI1XIx2rPQb9hlr8ZoXXz3W8/view?usp=drivesdk). Use the "genres" key to get the main genres and the "subgenres" key to get all subgenres and the "genre_map" to get the dict that has each main genres as keys and their subgenres as values.

    - The site is https://www.chosic.com/list-of-music-genres/, it has 5600+ total subgenres, I found another one that has 6200 genres tho but no subcategory breakdown. (https://everynoise.com/everynoise1d.html)

    - Also the offset doesn't let you get past the 1000 limit, it first has a range limit so the highest value of offset you can use is 1000 but if you are using Limit is 50, it's 950.

- He descargado el .json y voy a mapear los subgéneros a géneros de forma que podamos obtener más información!

In [4]:
import sys
sys.path.append("../")
import src.soporte_sql as sql
import pandas as pd
import ast
import json

# Tabla main_genres
- Tiene el nombre del género
- Tiene las veces que aparece (para el ranking)
- Conectado a la marca del que proviene

In [33]:
generos = pd.read_csv("../datos/01 Spotify/01 Resumen Marcas/resumen_zara.csv")
generos = generos[["brand","genres_ranking"]]
generos["genres_ranking"] = generos["genres_ranking"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

In [34]:
lista_brands = []
lista_generos = []
lista_conteos = []
for tuplas in generos["genres_ranking"]:
    for artista, conteo in tuplas:
        lista_generos.append(artista)
        lista_conteos.append(conteo)
        lista_brands.append(1) # 1 es zara en la base de datos

generos = pd.DataFrame({
    "subgenre_name" : lista_generos,
    "number_of_appearances" : lista_conteos,
    "brand_id" : lista_brands
})
generos.to_csv("../datos/02 Base de Datos/07_ranking_y_generos_zara.csv",index=False)
generos.head()

,subgenre_name,number_of_appearances,brand_id
0,lo-fi,1637,1
1,lo-fi beats,1235,1
2,afro house,1195,1
3,tech house,1112,1
4,classical,1075,1


### Usando el json, mapeamos los subgeneros a los géneros

In [35]:
with open("../datos/00_Spotify_Genres/genres_dict.json", "r") as file:
    genres_mapping = json.load(file)

In [36]:
dictio_genres = {}
for main_genre, subgenres in genres_mapping["genres_map"].items():
    for subgenre in subgenres:
        dictio_genres[subgenre] = main_genre

In [37]:
generos["genre"] = generos["subgenre_name"].map(dictio_genres)

In [ ]:
c = generos["genre"].isna() == True
generos[c].to_excel("../datos/00_Spotify_Genres/01_generos_no_identificados_zara.xlsx")


### Crear tabla y subir datos

In [ ]:
conexion = sql.conectar_bd()
query = ''' create table main_genres (
            id bigint primary key generated always as identity,
            genre_name text not null,
            number_of_appearances int not null,
            brand_id bigint references brands (id) on delete cascade
          );'''
sql.modificar_bd(conexion,query)

Conectado a la base de datos
Se ha modificado correctamente la base de Datos


In [ ]:
conexion = sql.conectar_bd()
query = '''INSERT INTO genres(genre_name,number_of_appearances,brand_id) VALUES (%s,%s,%s)'''
sql.insertar_muchos_datos(conexion,query,sql.generar_tupla(generos))

Conectado a la base de datos
Se han añadido los valores correctamente


# Tabla subgenres
- Tiene el nombre del subgénero
- Tiene las veces que aparece (para el ranking)
- Conectado del main_genre al que pertenece
- Conectado del brand al que pertenece
